In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

### 1. Formulate the statement of the interpolation problem with Cubic Spline [mathematical formula]

Задача интерполяции состоит в восстановлении непрерывной функции используя таблицу (сеточную проекцию некой функции $f(t)$.
Сеточная проекция определяется с помощь оператора рестрикции $R$ на отрезке $[a,b]$ таким образом:
$$
\begin{aligned}
 f_n = \{f(t_n)\}^N_{n=0}
\end{aligned}
$$
Каждый из узлов $t_n$ входит в **совокупность узлов**, которая в простейшем случае *равномерна*:
$$
\begin{aligned}
 \{t_n\}^N_{n=0}, t_n = a + n\tau, \tau = \frac{b-a}{N}, t \in [a,b] 
\end{aligned}
$$

**Сплайн** – кусочно-многочленная глобальная интерполяция, обозначаемая $S_m(t)$, где $m$ обозначает степень многочлена. В случае кубического сплайна $m=3$.

**Гладкостью** сплайна $l$ называется количество непрерывных производных в узлах сетки.

**Дефектом** сплайна называется разность $d=l-m$.

Итак, кубическим сплайном дефекта $d$, интерполирующим на отрезке $[a,b]$ заданную функцию $f(t)$, называют функцию $S(t)$ удовлетворяющую следующим условиям:

1. $S(t_n) = f(t_n)$ – условие интерполяции в узлах сетки $\{t_n\}^N_{n=0}$.

2. $S(t) \in C^l[a,b]$ – непрерывна вместе с $l=d+m$ первыми производными.

3. На каждом отрезке $[t_n, t_{n+1}]$ ($n = 0, \dots, N-1$), $S(t)$ является кубическим многочленом.

4. На краях отрезка $[a,b]$ заданы некоторые краевые условия на производные функции и сплайна.

Задача имеет единственное решение.

К примеру, кубический интерполянт дефекта 1 задается такой многозвенной функцией:

$S_3(t) = \bigcup_{n=0}^{N-1}S_{3,n}(t) \in C_2[a, b]$

### 2. Formulate the functional and differential compatibility conditions [mathematical formula]

$\delta S^{(p)}(t)|_{t = t_{n}} = S^{(p)}(t)|_{t = t_{n}} - f^{(p)}(t_{n}) = 0, n = 0, \dots, N-1$

Условие функционально, когда $p=0$, дифференциально, когда $p=2$.

### 3. Formulate stitching conditions [mathematical formula]

$S^{'}_{3, n-1}(t) \large|_{t=t_n} = S^{'}_{3, n}(t), \large|_{t=t_n}, n = 0, \dots, N-1$







### 4. Justify why these conditions provide you with the required smoothness [thesis text, no more than 500 characters]

Функциональное условие необходимо для соответсвия сплайна исходной функции в узлах сетки.

Дифференциальное условие обеспечивает гладкость решения, что обеспечивает удобство работы со сплайном и расширяет спектр возможных его применений. 

Условия сшивания обеспечиваю непрерывность решения.

Все это вместе позволяет гарантировать что функция и некоторое количество ее производных гладкие и непрерывные.

### 5. Derive dependency formula: the dependence of the second derivatives at the grid nodes on the increment of the function (the function values difference on the grid nodes). [Mathematical formulas derivation. Detailed, with clear transitions]

Сплайн:

$S_{3, n} = a_{0, n} + a_{1, n}(t - t_{n}) + a_{2, n}(t - t_{n})^{2} + t_{3, n}(t - t_{n})^{3}$

Первая производная сплайна:

$S^{'}_{3, n}(t) = a_{1, n} + 2a_{2, n}(t - t_n) + 3a_{3, n}(t - t_n)^2$

Вторая производная сплайна:

$S^{''}_{3, n}(t) = 2a_{2, n} + 6a_{3, n} (t - t_n)$

Запишем условия согласования:

$\left\{\begin{array} \\
S_{3, n}(t)|_{t=t_{n}} = f(t_{n})\\ 
S_{3, n}(t)|_{t=t_{n+1}} = f(t_{n+1})\\ 
S^{''}_{3, n}(t)|_{t=t_{n}} = f^{(2)}(t_{n})\\ 
S^{''}_{3, n}(t)|_{t=t_{n+1}} = f^{(2)}(t_{n+1})\end{array}\right.$

Подставим значения сплайна и производных:

$\left\{\begin{array} \\
a_{0, n} + a_{1, n}(t_{n} - t_{n}) + a_{2, n}(t_{n} - t_{n})^{2} +a_{3, n}(t_{n}- t_{n}) ^ {3} = f(t_{n})\\ 
a_{0, n} + a_{1, n}(t_{n+1} - t_{n}) + a_{2, n}(t_{n+1} - t_{n})^{2} + a_{3, n}(t_{n+  1} - t_{n})^{3} = f(t_{n+1})\\ 
2a_{2,n} + 6a_{3,n}(t_{n} - t_{n}) =f^{(2)}(t_{n})\\ 
2a_{2,n} + 6a_{3,n}(t_{n+1}-t_{n}) = f^{(2)}(t_{n+1})\end{array}\right.$

Пусть $f(t_{n}) = f_{n}$, $f^{''}(t_{n}) = m_{n}$, $t_{n+1} - t_{n} = \tau_{n+1}$. Решим систему относительно $a$:

$S_{3, n}(t) = f_{n} + (\frac{f_{n+1} - f_{n}}{\tau_{n+1}} - \frac{\tau_{n+1}}{2}m_{n} - \frac{\tau_{n+1}}{6}(m_{n+1} - m_{n}))(t-t_{n}) + \frac{m_{n}}{2}(t-t_{n})^{2} + \frac{1}{6\tau_{n+1}}(m_{n+1} - m_{n})(t-t_{n})^{3}, n = 0, \dots, N-1$.

Введем условия стыковки:
$\left\{\begin{array} \\
S'_{3, 0}(t)|_{t=t_{1}} = S'_{3, 1}(t)|_{t=t_{1}}\\ 
S'_{3, 1}(t)|_{t=t_{2}} = S'_{3, 1}(t)|_{t=t_{2}}\\ 
\dots \\
S'_{3, n-2}(t)|_{t=t_{n-1}} = S'_{3, n-1}(t)|_{t=t_{n-1}}\\ 
\end{array}\right.$

Подставим $S^{'}_{3, n}(t)$ и получим искомое:

$\frac{\tau_{n}}{6}m_{n-1} + \frac{\tau_{n} + \tau_{n +1}}{3}m_{n} + \frac{\tau_{n+1}}{6}m_{n+1} = \frac{f_{n+1} - f_{n}}{\tau_{n+1}} - \frac{f_{n} - f_{n-1}}{\tau_{n}} n = 0, \dots, N-1$.

### 6. Create a system of equations using this formula [Matrix representation. Mathematical formulas]

$\begin{pmatrix}
\frac{\tau_{1}}{6} & \frac{\tau_{1} + \tau_{2}}{3} & \frac{\tau_{2}}{6} &  0 & ... & ...\\ 
 0 & \frac{\tau_{2}}{6}& \frac{\tau_{2} + h_{3}}{3} & \frac{\tau_{3}}{6} & 0 & ...\\ 
 ... & ... & ... & ... & ... & ...\\ 
 ... & 0 & \frac{\tau_{n-2}}{6} & \frac{\tau_{n-2} + \tau_{n-1}}{3} & \frac{\tau_{n-1}}{6} & 0\\
... & ...  & 0 & \frac{\tau_{n-  1}}{6} & \frac{\tau_{n-1} +  \tau_{n}}{3} & \frac{\tau_{n}}{6}\\
\end{pmatrix}$ $\begin{pmatrix}
m_{0}\\ 
m_{1}\\ 
...\\ 
...\\ 
m_{n-1}\\ 
m_{n} \\
\end{pmatrix}$ = $\begin{pmatrix}
\frac{\Delta f_{1}}{\tau_{2}} - \frac{\Delta f_{0}}{\tau_{1}}\\ 
\frac{\Delta f_{2}}{\tau_{3}} - \frac{\Delta f_{1}}{\tau_{2}}\\ 
...\\ 
...\\ 
\frac{\Delta f_{n-2}}{\tau_{n-1}} - \frac{\Delta f_{n-3}}{\tau_{n-2}}\\ 
\frac{\Delta f_{n-1}}{\tau_{n}} - \frac{\Delta f_{n-2}}{\tau_{n-1}} \\
\end{pmatrix}$

где $\Delta f_{n} =  f_{n} -  f_{n-1}$






### 7. Explain what is an unknown variable in this system. whether the system is closed with respect to an unknown variable. What is missing for closure. [Text, no more than 200 characters]

Неизвестная переменная $m$ – вторая производная функции $f(t)$.

Система не замкнута по отношению к $m$, так как отсутсвуют краевые условия, которые необходимо задать неким образом, чтобы количество уравнений в системе было достаточным (необходимо добавить 2 уравнения).

### 8. Bring this matrix to the appropriate form to use the Tridiagonal matrix algorithm [Mathematical derivation. Use Gauss Elimination]

$\begin{pmatrix}
-b_{1} & c_{1} &  0 & ... & ...\\ 
a_{1} & -b_{2} & c_{2} & 0 & ...\\ 
 ... & ... & ... & ... & ... \\ 
 ... & 0 & a_{n-1} & -b_{n-1} & c_{n}\\
... & ...  & 0 & a_{n} & -b_{n}\\
\end{pmatrix} \hat{x} = \begin{pmatrix}
d_{1}\\ 
d_{2}\\ 
...\\ 
d_{n-1}\\ 
d_{n}\\
\end{pmatrix}$

Пусть $P_{1} = \frac{c_{1}}{b_{1}}$, $Q_{1} = -\frac{d_{1}}{b_{1}}$. Делим первую строчку на $-b_{1}$ и подставляем:

$\begin{pmatrix}
1 & -P_{1} &  0 & ... & ...\\ 
a_{1} & -b_{2} & c_{2} & 0 & ...\\ 
 ... & ... & ... & ... & ... \\ 
 ... & 0 & a_{n-1} & -b_{n-1} & c_{n}\\
... & ...  & 0 & a_{n} & -b_{n}\\
\end{pmatrix} \hat{t} = \begin{pmatrix}
Q_{1}\\ 
d_{2}\\ 
...\\ 
d_{n-1}\\ 
d_{n}\\
\end{pmatrix}$

Вычитаем первую строку из второй:

$\begin{pmatrix}
1 & -P_{1} &  0 & ... & ...\\ 
0 & a_{2}P_{1} -b_{2} & c_{2} & 0 & ...\\ 
 ... & ... & ... & ... & ... \\ 
 ... & 0 & a_{n-1} & -b_{n-1} & c_{n}\\
... & ...  & 0 & a_{n} & -b_{n}\\
\end{pmatrix} \hat{t} = \begin{pmatrix}
Q_{1}\\ 
d_{2} - a_{2}Q_{1}\\ 
...\\ 
d_{n-1}\\ 
d_{n}\\
\end{pmatrix}$

Делим вторую строку на $-b_{2} + a_{2}P_{1}$:

$\begin{pmatrix}
1 & -P_{1} &  0 & ... & ...\\ 
0 & 1 & \frac{c_{2}}{a_{2}P_{1} -b_{2}} & 0 & ...\\ 
 ... & ... & ... & ... & ... \\ 
 ... & 0 & a_{n-1} & -b_{n-1} & c_{n}\\
... & ...  & 0 & a_{n} & -b_{n}\\
\end{pmatrix} \hat{x} = \begin{pmatrix}
Q_{1}\\ 
\frac{d_{2} - a_{2}Q_{1}}{a_{2}P_{1} -b_{2}}\\ 
...\\ 
d_{n-1}\\ 
d_{n}\\
\end{pmatrix}$

Аналогично $P_{2} = -\frac{c_{2}}{a_{2}P_{1} -b_{2}}$, $Q_{2} = \frac{d_{2} - a_{2}Q_{1}}{a_{2}P_{1} -b_{2}}$.

$\begin{pmatrix}
1 & -P_{1} &  0 & ... & ...\\ 
0 & 1 & -P_{2} & 0 & ...\\ 
 ... & ... & ... & ... & ... \\ 
 ... & 0 & a_{n-1} & -b_{n-1} & c_{n}\\
... & ...  & 0 & a_{n} & -b_{n}\\
\end{pmatrix} \hat{t} = \begin{pmatrix}
Q_{1}\\ 
Q_{2}\\ 
...\\ 
d_{n-1}\\ 
d_{n}\\
\end{pmatrix}$

После Гауссовской эллиминации:

$t_{n-1} = P_{n-1}t_{n} + Q_{n-1}$

где:

$P_{1} = \frac{c_{1}}{b_{1}}$

$Q_{1} = -\frac{d_{1}}{b_{1}}$

$P_{2} = -\frac{c_{2}}{a_{2}P_{1} -b_{2}}$

$Q_{2} = \frac{d_{2} - a_{2}Q_{1}}{a_{2}P_{1} -b_{2}}$

$P_{n-1} = -\frac{c_{n-1}}{a_{n-1}P_{n-2} + b_{n-1}}$

$Q_{n-1} = \frac{d_{n-1} - a_{n-1}Q_{n-2}}{a_{n-1}P_{n-2} - b_{n-1}}$. 


### 9. Derive formulas of direct pass and reverse pass of Tridiagonal matrix algorithm [Mathematical formals]

Прямой проход определен в восьмом задании.


Обратный проход решается подстановкой $t_{n-1}$ в $t_{n-1} = P_{n-1}t_{n} + Q_{n-1}$ из прошлого задания.

Получим: 

$a_{n}(P_{n-1}t_{n} + Q_{n}) + b_{n} t_{n} = d_{n}$.

Продолжим решение с  $t_{n}$ и после с  $t_{n+1}$ и получим делаемый результат.

### 10. Implement code prototype of the future algorithm implementation. Classes/methods (if you use OOP), functions. The final implementation (on language chosen by you) should not differ from the functions declared in the prototype. [Python code]

In [1]:
def cubic_spline(X, Y):
  """
  Interpolate a cubic spline with deffect 1.
  
  :params:
  X: grid
  Y: values of function in grid nodes
  
  :return:
  S: spline function
  """
  pass

def tridiagonal_matrix_solve(A, d):
  """
  Solve tridiagonal matrix (Ax = d).
  
  :params:
  A: matrix
  d: vector
  
  :return:
  x: solution vector
  """
  pass

### 11. Derive formula of Cubic Spline method error [Mathematical formulas]

Выводы данных формул не приведены в учебниках, но есть в очень обширной форме (включая доказательства нескольких лемм и теорем) в найденной научной статье.

Все формулы посчитаны с учетом равномерной сетки, однако могут быть применены и для неравномерного случая при выборе длины макимального промежутка между узлами для расчетов.

Пусть $M_4 = \max_{[a,b]}|f^{(4)}(t)|$.

Оценка ошибки:

$|| f^{(p)}(t) - S_3^{(p)}(t) ||_{C[a,b]} = \max \limits_{[a,b]} |  f^{(p)}(t) - S_3^{(p)}(t) | \leq M_4 h^{4  - p}, p = 0, 1, 2$


### 12. Rate the complexity of the algorithm [Text, and rate in terms of big O, no more than 100 characters]

Алгоритм требует два прохода – прямой и обратный.

Каждый из проходов имеет сложность $O(n)$, таким образом **сложность алгоритма $O(n)$**.

### Congrats!